<a href="https://colab.research.google.com/github/Alookapola/pytorch-tutorial/blob/master/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 设置路径
tr_path = 'covid.train.csv'  # path to training data
tt_path = 'covid.test.csv'   # path to testing data

!gdown --id '19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF' --output covid.train.csv
!gdown --id '1CE240jLm2npU-tdz81-oVKEF3T2yfT1O' --output covid.test.csv

Downloading...
From: https://drive.google.com/uc?id=19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF
To: /content/covid.train.csv
100% 2.00M/2.00M [00:00<00:00, 44.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CE240jLm2npU-tdz81-oVKEF3T2yfT1O
To: /content/covid.test.csv
100% 651k/651k [00:00<00:00, 21.6MB/s]


In [10]:
# test 
with open(tr_path, 'r') as fp:
  data = list(csv.reader(fp))
  data = np.array(data[1:])[:, 1:].astype(float)
data.shape
# (2700, 94)



(2700, 94)

In [4]:
# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader ##

# For data processing
import numpy as np
import csv
import os

# Plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

myseed = 42069
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)




In [6]:
def get_device(): 
  return 'cuda' if torch.cuda.is_available() else 'cpu'
get_device()

def plot_learning_curve(loss, title=''):
  total_steps = len(loss['train'])


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):

'cpu'

In [23]:
# 给dataset建模一个class
class COVID19Dataset(Dataset):
  def __init__(self, path, mode='train', target_only=False):
    self.mode = mode

    with open(path, 'r') as fp:
      data = list(csv.reader(fp))  #$$
      data = np.array(data[1:])[:, 1:].astype(float) # 排除第一列之后读取
    
    if not target_only:
      feats = list(range(93))  # 全部列的序号

    else:
      feats = list(range(40))
      feats.append(57)
      feats.append(75)

    if mode == 'test':
      #  Testing data
      # data: 893 x 93 (40 states + day 1 (18) + day 2 (18) + day 3 (17))
      data = data[:, feats]
      self.data = torch.FloatTensor(data)

    else:
      # Training data (train/dev sets)
      # data: 2700 x 94 (40 states + day 1 (18) + day 2 (18) + day 3 (18))
      target = data[:, -1]
      data= data[:, feats]

      if mode == 'train':
        idx = [i for i in range(len(data)) if i % 10 != 0]
      elif mode == 'dev':
        idx = [i for i in range(len(data)) if i % 10 == 0] # 验证集

      # tensors转化                          
      self.data = torch.FloatTensor(data[idx])
      self.target = torch.FloatTensor(target[idx])

    # Normalize features (you may remove this part to see what will happen)特征标准化 z-score标准化 同一个feature不同人之间
    self.data[:, 40:] = \
    (self.data[:, 40:] - self.data[:, 40:].mean(dim=0, keepdim=True))/self.data[:, 40:].std(dim=0, keepdim = True)

    self.dim = self.data.shape[1] #横向的编号是1 这个方向的维度

    print('Finished reading the {} set of COVID19 Dataset ( {} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))
    
  #
  def __getitem__(self, index):
    if self.mode in ['train', 'dev']:
      return



IndentationError: ignored

In [24]:
a = list(range(40))
a.append(57)
a.append(75)
#a.append(57).append(75)


**DATALOADER**

In [ ]:
def prep_dataloader(path, mode, batch_size, n_jobs=0, target_only=False):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = COVID19Dataset(path, mode=mode, target_only=target_only)  # Construct dataset
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'), drop_last=False,  # 只在training时shuffle
        num_workers=n_jobs, pin_memory=True)                            # Construct dataloader
    return dataloader

In [ ]:
class NeuralNet(nn.Nodule):
  def __init__(self, input_dim):
    super(NeuralNet, self).__init__()  # 利用父类initialization

    # Define the Network
    self.net = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 1)
    )

    # Mean squared error loss
    self.criterion = nn.MSELoss(reduction = 'mean')  # torch.nn中的功能

  
  def forward(self, x):
    return self.net(x).squeeze(1)

  def cal_loss(self, pred, target):
    return self.criterion(pred, target)

Training

In [1]:
def train(tr_set, dv_set, model, config, device): # 注意这里的model作为输入
  
  n_epochs = config['n_epochs'] # 总共的epochs数

  optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas']) # 优化器 默认SGD
  min_mse = 1000.
  loss_record = {'train':[], 'dev':[]}
  early_stop_cnt = 0
  epoch = 0

  while epoch < n_epochs:
    model.train()  # 设置为训练模式
    for x, y in tr_set:
      optimizer.zero_grad()
      x, y = x.to(device), y.to(device)  # 将data移动到device中
      pred = model(x)
      mse_loss = model.cal_loss(pred, y)
      mse_loss.backward()
      optimizer.step()  # update model with optimizer
      loss_record['train'].append(mse_loss.detach().cpu().item())
      # detach阻断反向传播 也可以用mse_loss().detach().cpu().numpy()

    # After each epoch, test your model on the validation (development) set.
    dev_mse = dev(dev_set, model, device)  # 主义这里的dev函数, 在后面定义
    if dev_mse < min_mse:
      # 保留改进之后的模型
      min_mse = dev_mse

      torch.save(model.state_dict(), config['save_path'])
      early_stop_cut = 0 # 重新计数
    
    else:
      early_stop_cnt += 1

    epoch += 1
    loss_record['dev'].apppend(dev_mse)
    if early_stop_cnt > config['early_stop']: # 很多步没有改进之后，跳出循环
      break
    
  print('Finished training after {} epochs'.format(epoch))
  return min_mse, loss_record # 优化过程中最小的损失函数值 and 训练中loss_function的值变化


SyntaxError: ignored

Validation

In [ ]:
def dev(dv_set, model, device):
  model.eval() # 调整到evaluation模式
  total_loss = 0
  for x, y in dv_set:
    x, y = x.to(device), y.to(device)
    with torch.no_grad(): # 不计算梯度
      pred = model(x)
      mse_loss = model.cal_loss(pred,y)
    total_loss += mse_loss.detach().cpu().item()


In [ ]:
# 一系列超参数
device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = False                   # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 270,               # mini-batch size for dataloader
    'optimizer': 'SGD',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.001,                 # learning rate of SGD
        'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth'  # your model will be saved here
}